In [5]:
import mat73
import matplotlib.pyplot as plt
import numpy as np
from tqdm import notebook
from scipy import stats
from scipy.signal import hilbert
from scipy.signal import butter, sosfiltfilt
from scipy import signal
from scipy.fft import fftshift
import pickle
import joblib
import os

## define bandpower func

In [6]:
def bandpower(data, fs, band, method, window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * fs
        else:
            nperseg = (2 / low) * fs

        freqs, psd = welch(data, fs, nperseg=nperseg)
        
    else:
        print('Method not implemented.')

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [7]:
data_dir = r'../processed_data'

## load lfp (mat 7.3) file

In [8]:
filepath = 'lfp.mat'
ey_lfp = mat73.loadmat(os.path.join(data_dir, filepath))
fs = int(ey_lfp["lfp"]["samplerate"][0])
print(f'sampling frequency is {fs} Hz')
full_lfp = np.vstack([ey_lfp['lfp']['data'][ch] for ch in range(len(ey_lfp['lfp']['data']))]).T
print(f'full LFP lasted for {full_lfp.shape[0]/(fs*60*60)} hours across {full_lfp.shape[1]} channels')

sampling frequency is 1250 Hz
full LFP lasted for 4.540507555555555 hours across 39 channels


(18000000, 12)

In [9]:
full_lfp_reduced_channels = full_lfp[:, [3,4,5,1,35,37,19,24,29,8,13,16]]
full_lfp_reduced_channels.shape

(20432284, 12)

In [ ]:
def lfp_bp_extract(lfp, offset):
    # define frequency ranges for band powers
    freq_bands = [[3, 8], [10, 30], [30, 50], [50, 80]]
    bands = ['theta', 'beta', 'lower_gamma', 'higher_gamma']
    # initialize list
    band_pow = []

    # define window in seconds for frequency analysis
    window_dur = 1
    # define time total in hours for analysis
    duration_total = 4

    win_size = int(window_dur*fs) # sampling at 1250 Hz, 1s is 1250 samples
    offset_size = int(offset*fs) # offset is in secs
    nun_overlaps = 1
    win_num = int(duration_total*60*60/window_dur*nun_overlaps)

    # for each band
    for _, freq_band in enumerate(notebook.tqdm(freq_bands)):
        instant_band_pow_all = []
        # for each channel
        for ch in notebook.tqdm(range(lfp.shape[1])):
            instant_band_pow_single = []
            # for each analysis window
            for win in notebook.tqdm(range(win_num)): 
                # window will be minimum nyquist 2/low_cut
                bp = bandpower(lfp[int(offset_size+win*win_size*1/nun_overlaps):int(offset_size+win*win_size*1/nun_overlaps)+win_size, ch],
                                        fs, 
                                        freq_band, 
                                        'welch')
                # a list of instantaneous single band power single channel
                instant_band_pow_single.append(bp) 
            # a list of 39 channels, single band power
            instant_band_pow_all.append(np.hstack((instant_band_pow_single)))
        band_pow.append(instant_band_pow_all)
    
    features = np.vstack(([np.vstack(band_pow[b]) for b in range(len(band_pow))]))
    zscore_feats = stats.zscore(features, axis=1)
    zscore_feats.T.shape
    pickle.dump(zscore_feats, open(f'../processed_data/welch_bp/welch_nonoverlap_zscore_bp_{offset}_s_offset.np', 'wb'))

In [ ]:
for offset in range(0.1, 1, 0.1):
    lfp_bp_extract(full_lfp_reduced_channels, offset)